In [1]:
import os
import geopandas as gpd
import pandas as pd

In [2]:
import sys
sys.path.append('../')
from process.append import get_enriched_features

In [3]:
california_boundary = gpd.read_file(r'D:\WORK\wildfire\Interagency-Tracking-System\its\Interagency Tracking System.gdb', 
                                    driver='OpenFileGDB', 
                                    layer='California')

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  return ogr_read(


In [4]:
enriched_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0"

In [5]:
enriched_list = os.listdir(enriched_path)

In [6]:
append_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb"

In [7]:
# skip the append, reports gdb file path
skip_list = ['appended.gdb', 'reports.gdb']

In [8]:
point_layers = []
line_layers = []
poly_layers = []
for f in enriched_list:
    if f in skip_list:
        continue
    f_path = enriched_path + r"\{}".format(f)
    gdb_layers = gpd.list_layers(f_path)
    for i in range(len(gdb_layers)):
        if 'point' in gdb_layers.loc[i, 'geometry_type'].lower():
            point_layers.append({'gdb_path': f_path, 'layer_name':gdb_layers.loc[i, 'name']})
        elif 'line' in gdb_layers.loc[i, 'geometry_type'].lower():
            line_layers.append({'gdb_path': f_path, 'layer_name':gdb_layers.loc[i, 'name']})
        else:
            poly_layers.append({'gdb_path': f_path, 'layer_name':gdb_layers.loc[i, 'name']})
            
enriched_layers = {'point': point_layers, 
                  'line': line_layers,
                  'polygon': poly_layers}

In [10]:
# one time change for activity category

In [11]:
pfirs_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\PFIRS_2020_2025.gdb"
layer_name = 'PFIRS_20250402'

In [12]:
enriched_layers['point'].append({'gdb_path': pfirs_path, 'layer_name': layer_name})

In [23]:
enriched_polygons, enriched_lines, enriched_points = get_enriched_features(enriched_layers)

2025-06-04 16:16:59,791 INFO  [process.append_polygon]  --------------------------------------------------------------------------------
2025-06-04 16:16:59,791 INFO  [process.append_polygon]  Concatenate all polygon records
2025-06-04 16:16:59,792 INFO  [process.append_polygon]  Load GeoDataFrame from the layer 'BLM_enriched_20250509' in 'D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\BLM_1950_2025.gdb' 
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
2025-06-04 16:16:59,890 INFO  [process.append_polygon]  Load GeoDataFrame from the layer 'CNRA_enriched_20250512_polygon' in 'D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\CNRA_1950_2025.gdb' 
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
2025-06-04 16:17:03,336 INF

In [16]:
import dask_geopandas
# some data need to be converted to multi-type again
from utils.save_gdf_to_gdb import save_gdf_to_gdb

In [15]:
# grab timber non spatial path again
timber_nonspatial_path = None
for p in enriched_layers['point']:
    if 'Timber_Nonspatial' in p['gdb_path']:
        timber_nonspatial_path = p
        break

In [24]:
%%time
for df, lyr_name in zip([enriched_polygons,enriched_lines,enriched_points], ["appended_poly","appended_line","appended_point"]):
    # init dask gdf for multithread clipping
    ddf = dask_geopandas.from_geopandas(df, npartitions=16)
    # clip to california bounds
    append_clipped = ddf.sjoin(california_boundary, how='inner', predicate='intersects').compute()
    # drop unwanted artifact columns from California boundary df
    append_clipped = append_clipped.drop(['index_right', 'Shape_Area', 'Shape_Length'], axis=1)
    
    # industry nonspatial is by design out of california bounds and got clipped, manually concat it back
    if lyr_name == 'appended_point':
        timber_nonspatial = gpd.read_file(timber_nonspatial_path['gdb_path'], 
                                    driver='OpenFileGDB', 
                                    layer=timber_nonspatial_path['layer_name'])
        
        append_clipped = pd.concat([append_clipped, timber_nonspatial], ignore_index=True)
    save_gdf_to_gdb(append_clipped, append_path, lyr_name)

2025-06-04 16:45:12,203 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-06-04 16:45:12,204 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-06-04 16:45:12,234 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb


C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field index_right of type Integer64 will be written as a Float64. To get Integer64, use layer creation option TARGET_ARCGIS_VERSION=ARCGIS_PRO_3_2_OR_LATER
  ogr_write(
2025-06-04 16:45:42,840 INFO  [pyogrio._io           ]  Created 441,964 records
2025-06-04 16:45:50,028 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb appended_poly
2025-06-04 16:53:33,698 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-06-04 16:53:33,699 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-06-04 16:53:33,709 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb


C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field index_right of type Integer64 will be written as a Float64. To get Integer64, use layer creation option TARGET_ARCGIS_VERSION=ARCGIS_PRO_3_2_OR_LATER
  ogr_write(
2025-06-04 16:53:55,149 INFO  [pyogrio._io           ]  Created 131,654 records
2025-06-04 16:54:04,957 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb appended_line
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
<timed exec>:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
<timed exec>:12

D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb


2025-06-04 16:54:07,643 INFO  [pyogrio._io           ]  Created 17,357 records
2025-06-04 16:54:07,760 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb appended_point


CPU times: total: 1h 4min 17s
Wall time: 35min 57s


In [36]:
enriched_polygons = gpd.read_file(append_path, driver='OpenFileGDB', layer='appended_poly')
enriched_points = gpd.read_file(append_path, driver='OpenFileGDB', layer='appended_point')
enriched_lines = gpd.read_file(append_path, driver='OpenFileGDB', layer='appended_line')

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [26]:
from datetime import datetime

In [27]:
def get_activity_report(enriched_points, enriched_lines, enriched_polygons):
    append_all = pd.concat([enriched_lines, enriched_points, enriched_polygons])
    append_all = append_all[(append_all['COUNTS_TO_MAS'] == 'YES')]
    
    append_all.geometry = gpd.points_from_xy(append_all['LONGITUDE'],append_all['LATITUDE'])
    
    
    append_all = append_all[["AGENCY",
        "ADMINISTERING_ORG",
        "PRIMARY_OWNERSHIP_GROUP",
        "COUNTY",
        "REGION",
        "ACTIVITY_DESCRIPTION",
        "ACTIVITY_CAT",
        "BROAD_VEGETATION_TYPE",
        "ACTIVITY_STATUS",
        "ACTIVITY_QUANTITY",
        "ACTIVITY_UOM",
        "ACTIVITY_END",
        "Year_txt",
        "geometry"
    ]]
    
    # check if geometry is_valid
    # personally this is redundent, unless lat, lon is not valid, but that would throw an error in previous part
    append_all = append_all[append_all.is_valid]
    
    
    def get_entity_type(agency):
        if agency in ['CALEPA', 'CALSTA', 'CNRA', 'PARKS', 'California State Parks']:
            return 'State'
        if agency in ['DOD', 'DOI', 'USDA', 'DOE', 'NPS']:
            return 'Federal'
        if agency in ['Industrial Timber', 'Timber Companies', 'TIMBER']:
            return 'Timber Companies'
        else:
            return None
        
        
    append_all['ENTITY_TYPE'] = append_all['AGENCY'].apply(get_entity_type)
    
    return append_all

In [28]:
activity_report_gdf = get_activity_report(enriched_points, enriched_lines, enriched_polygons)

In [29]:
save_gdf_to_gdb(activity_report_gdf, 
                r'D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\reports.gdb', 
                f'activity_report{datetime.today().strftime('%Y%m%d')}')

2025-06-04 16:55:19,868 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-06-04 16:55:19,868 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-06-04 16:55:19,898 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\reports.gdb


2025-06-04 16:55:25,458 INFO  [pyogrio._io           ]  Created 489,897 records
2025-06-04 16:55:28,365 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\reports.gdb activity_report20250604
